# openCypher Workshop - Section 03: Inserting and Updating Data

Before we get too far ahead, we should probably discuss how to insert and update data within Amazon Neptune.  There are also some best practices around loading data transactionally into Neptune in order to provide for high concurrency and throughput.  In this notebook we will cover the steps necessary to insert data, update (or upsert) data, and how to write queries for semi-bulk operations.

## Graph Model

As we have in previous notebooks, we've included here a copy of the graph data model for this dataset as reference to aid in writing your queries.

<img src="https://ee-assets-prod-us-east-1.s3.amazonaws.com/modules/f3f89ef4607743429fb01ae23d983197/v1/workshop/images/datamodel-updated.png"/>



## Create a node with properties

When using Gremlin or SPARQL, Neptune allows you to supply your own String IDs when creating vertices and edges. When using these languages you should always try to supply your own IDs, as finding an object by its ID is generally faster than trying to find it by property.

However, openCypher does not allow you to set the ID value unless it's through a [bulk load](https://docs.aws.amazon.com/neptune/latest/userguide/bulk-load-tutorial-format-opencypher.html). When creating vertices and nodes transactionally with openCypher, Neptune will create a String-based UUID for you. **\[insert best practices for handling ids in oc\]**

### 03.01 - Creating a node with CREATE

Below, we'll create a new node with id, firstName, and lastName properties using the `CREATE` clause. We've also included a traversal prior to the code block with the `DELETE` clause. We will cover this in more detail later, but this command is to ensure that the Person node with the property key of "id" and property value of "person0" does not exist before creating your new "person0" node.

Additional documentation for this new clause:
- `CREATE` - https://s3.amazonaws.com/artifacts.opencypher.org/openCypher9.pdf, page 99

In [ ]:
%%oc

MATCH (n {id: 'person0'})
DELETE n

We can now create a new node with a label of `Person` and properties for `firstName` and `lastName`. Since we're creating this node transactionally, Neptune will assign a UUID for its ID. We'll add an `id` property to help us easily identify this node later on.

In [ ]:
%%oc

CREATE (n:Person {id: 'person0', firstName: 'Lorem', lastName: 'Ipsom'})
RETURN n

### 03.02 - Overwriting property values and labels with `SET`

Using `SET` in openCypher lets you overwrite or create new property values.

- `SET` - https://s3.amazonaws.com/artifacts.opencypher.org/openCypher9.pdf, page 107

Let's see an example of this by first creating a node representing Sherlock Holmes:

In [ ]:
%%oc

CREATE (n:Artist {name: 'Sherlock Holmes', birthyear: 1899})
RETURN n

Actually, we made a mistake. Sherlock Holmes, by most accounts, was born in 1854, not 1899. So we need to correct his birthday.  He also died in 1893, so let's make sure we add that as well.

In [ ]:
%%oc 

MATCH (n:Artist {name: 'Sherlock Holmes'})
SET n.birthyear = 1854
SET n.deathyear = 1893
RETURN n

From the previous output, we were able to update a property with `SET` as well as use it to create new properties. 

### 03.03 Create a relationship between two nodes

Now that we have created a couple of nodes, let's try to create a relationship between them.  We do this using the `CREATE` clause again.

For reference:

- `CREATE` - https://s3.amazonaws.com/artifacts.opencypher.org/openCypher9.pdf, page 99

In the following traversal, we will add a `follows` relationship between Sherlock Holmes and the node representing you:

In [ ]:
%%oc

MATCH (n:Artist {name: 'Sherlock Holmes'})
MATCH (m:Person {firstName: 'Lorem', lastName: 'Ipsom'})
CREATE (n)<-[e:follows]-(m)
RETURN e

The returned result shows information about the relationship created between Sherlock Holmes and the node representing you.

Let's check our work with a pattern that finds the nodes connected to us, via a `follows` edge:

In [ ]:
%%oc

MATCH (n:Person {firstName: 'Lorem', lastName: 'Ipsom'})-[:follows]->(m)
RETURN m

You should see all of the properties associated with Sherlock Holmes.

### 03.04 - Conditional writes with MERGE

Sometimes, you'll want upsert-like functionality, whereby you create a new node or relationship only if it doesn't already exist. If it already exists, you use the existing instance. 

You can achieve this functionality by using the `MERGE` clause. When using `MERGE`, the clause will check if the specified pattern already exists in the graph. If it does, it will bind to those patterns, and if it doesn't, it will create them. 

Neptune asserts uniqueness for node IDs and relationship IDs only. No two nodes can have the same ID, nor can any two relationships can have the same ID (a node and a relationship, however, can have the same ID). However, there may be a situation where you don't want two nodes to have the same label and/or property values, and if that's the case, `MERGE` is a useful way of enforcing this.

Additional documentation:

- `MERGE` - https://s3.amazonaws.com/artifacts.opencypher.org/openCypher9.pdf, page 115

Let's see this in more detail.  For example, what happens if we were to attempt to create another vertex with ID 'person0' (which is already in our graph)?  Give the following code block a try.

In [ ]:
%%oc

MERGE (n:Person {id: 'person0', firstName: 'Lorem', lastName: 'Ipsom'})
RETURN n

The above query only outputs one matching node. When we used `MERGE`, the pattern we defined already exists in the graph (from the node we created earlier in this notebook), so instead of creating a new instance, it simply uses the instance it found.

Just to keep us honest, let's try adding a new vertex with this type of traversal pattern.  Here we will check for the existance of a `Person` node with the given properties.  We'll use the `MERGE` clause to create the node if it doesn't exist:

In [ ]:
%%oc 

MERGE (n:Person {id: 'person1', firstName: 'John', lastName: 'Doe'})
RETURN n

`MERGE` can also be used to match or create a relationship, like so:

In [ ]:
%%oc

MATCH (n:Person {id: 'person1'}),
      (m:Artist {name: 'Sherlock Holmes'})
MERGE (n)-[e:follows]->(m)
RETURN e

### 03.05 - Bulk write operations

Last, but not least, you can also create multiple nodes and edges by separating them with a comma within the CREATE clause.

In the following example, we will create a small sample graph with 5 nodes and 5 relationships. 

In [ ]:
%%oc

CREATE (mgm:Hotel {name: 'MGM'}),
       (aria:Hotel {name: 'Aria'}),
       (venetian:Hotel {name: 'Venetian'}),
       (wynn:Hotel {name: 'Wynn'}),
       (encore:Hotel {name: 'Encore'}),
       (mgm)-[:`Bus Route`]->(aria),
       (aria)-[:`Bus Route`]->(venetian),
       (mgm)-[:`Bus Route`]->(wynn),
       (encore)-[:`Bus Route`]->(aria),
       (venetian)-[:`Bus Route`]->(mgm)

Let's check our work:

In [ ]:
%%oc

MATCH (n:Hotel)
RETURN n

In [ ]:
%%oc

MATCH ()-[e:`Bus Route`]-()
RETURN e

### Review

In this notebook we covered the different ways that you can create and update data within Neptune using openCypher.  We also covered some of the best practices regarding how to use vertex/edge IDs and how to batch multiple writes into a single transaction.

### End of Section 03 - Move onto the next notebook.